# Capacitated Vehicle Routing Problem (CVRP)

A CVRP in an academic sense is to find the least number of vehicles which are required to service all demands at minimum cost. The way in which we emulate this is to add a large cost for using a vehicle and a smaller cost for travelling between nodes. Each vehicle is limited to the maximum amount that it can service (referred to as the capacity) and each node has a defined `quantity` for it's demand.

## Requirements
* **Protobuf** (Google's Protocol Buffers library) (https://pypi.org/project/protobuf/) `pip install protobuf`
* **Requests** (HTTP Python library) `pip install requests`
* **Plotting** (ipyleaflet) [installation instructions here](https://github.com/jupyter-widgets/ipyleaflet)
* Icepack API token of a key that has the Travelling Salesman Problem Solver enabled
* See the TSP/TSPTW examples for more details. If you can run those, you can run this example :-) 

## Schema details
If you're looking for more details around the schema fields and complete explanations check out the documentation on docs.scrudu.io



### Load the data
We're going to load some sample data which already has some mock data around demands at each node. We're going to make a fixed demand at each node of 20 units.

In [ ]:
import pandas
df = pandas.read_csv('../../sample_data/publist.csv').head(10) # just grabbing the top 10 points.
df['quantity'] = 20       # create a quantity field of 20 units per node
df.loc[0,'quantity'] = 0  # set the quantity of the first node to zero (the depot)
print(df.head())


In [ ]:
exec(open('apiHelper.py').read()) # import some helper classes which we've written for you.

api = apiHelper(modelType="cvrp-jkfdoctmp51n") # set the model type to the cvrp model

sr = cvrp_jkfdoctmp51n_pb2.SolveRequest()
sr.solveType = 0

m = sr.model

def makePoint(row):
    p = cvrp_jkfdoctmp51n_pb2.Geocode()
    p.id = row.id
    p.x = row.X
    p.y = row.Y
    p.quantity  = row.quantity
    return (p)

# set the depot in the model
m.depot.CopyFrom(makePoint(df.loc[0,]))

# Add the balance of the data
for index, row in df.iterrows():
    if index != 0:
        m.points.append(makePoint(row))
        
m.NumberOfVehicles = 3
m.VehicleCapacity = 100
m.distancetype = 1

print(m) # that's what the whole model looks like.


### Model summary
So we've created a CVRP model with 9 deliveries, a designated depot, three vehicles and a capacity of 100 on each vehicle. This model assumes a homogeneous fleet (all the vehicles are the same) - if you require modelling the specifics of each vehicle you can use the IVR7/8 Model. 
It should be obvious that with a capacity of 100 on each vehicle, the maxaimum number of stops per vehicle is 5. Given we have 9 stops, we know then that this can be completed with just two vehicles (i.e. the third vehicle need not be used if our first goal is to minimise the number of vehicles used).

### Run the model
We can now submit the model to the api using the api-helper object.

In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Retrieve the model response
We can use the api-helper to grab the solution response using the request-id provided by the post.

In [ ]:
sol = api.Get(reqId)

print(sol.objective) # This is the cost of the solution returned by the api.


### Visualising the response
The format that is returned by the api is similar to the TSP, except that each "tour" is organised by vehicle. This means that we have a list of routes and within each route we have the geometry that applies to the relevant stops therein.

In [ ]:

r = pandas.DataFrame(columns=['vehicleId', 'stop', 'QtyDelivered'])
g = list() # for storing the geometries
idx = 0
for i, e in enumerate(sol.routes):
    for j, s in enumerate(e.sequence):
        r.loc[idx] = list([i, s, e.visitCapacities[j]])
        idx+=1
    gs = list()
    for _, f in enumerate(e.edges):
        for _, p in enumerate(f.geometry):
            gs.append([p.y, p.x])
    g.append(gs)

# we can then left-join on the original data.    
r = pandas.merge(r,df,left_on='stop',right_on='id', how = 'left').drop(['id','quantity'], axis=1)

print(r)



In [ ]:
from ipyleaflet import Map, Polyline, Circle, LayerGroup
cvec = list(['blue', 'red', 'orange'])

# form some locations to add to the map (just green dots seems ok)
locs = list()
for index, p in r.iterrows():
    circle = Circle()
    circle.location = (p['Y'], p['X'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

center = [r['Y'][1],r['X'][1]] # just use the first point as the center (i.e. the depot)
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
for i, gs in enumerate(g):
    m.add_layer(Polyline(locations=gs,color=cvec[i],fill=False))

m.add_layer(LayerGroup(layers=(locs)))
m


## Request Statistics
The statistics for all recent requests, including the solve duration, can be viewed in the dashboard of the [Icepack Client Portal](https://portal.icepack.ai/dashboard).